In [ ]:

from sklearn.ensemble import VotingClassifier,RandomForestClassifier,VotingRegressor, RandomForestRegressor,StackingRegressor
from sklearn.pipeline import Pipeline
from sklearn.svm import SVR
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder ,MinMaxScaler,StandardScaler,RobustScaler
from sklearn.model_selection import train_test_split, cross_val_score,GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error,r2_score,accuracy_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn import tree
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from sklearn.impute import SimpleImputer

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/mujahidashraf/data/refs/heads/main/insurance.csv")

In [ ]:
le = LabelEncoder()
df['sex'] = le.fit_transform(df['sex'])
df['smoker'] = le.fit_transform(df['smoker'])
df['region'] = le.fit_transform(df['region'])
X = df[['age','sex','bmi','children','smoker','region']]
y = df['expenses']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
pipeline_lr = Pipeline([
    ('scaler', StandardScaler()),
    ('lr', LinearRegression())
])

pipeline_ridge = Pipeline([
    ('scaler', StandardScaler()),
    ('ridge', Ridge())
])

pipeline_lasso = Pipeline([
    ('scaler', StandardScaler()),
    ('lasso', Lasso())
])

pipeline_knn = Pipeline([
    ('scaler', StandardScaler()),
    ('knn', KNeighborsRegressor(n_neighbors=5))
])

pipeline_tree = Pipeline([
    ('scaler', StandardScaler()),
    ('tree', DecisionTreeRegressor(ccp_alpha=0.0,
        criterion='squared_error',
        max_depth=5,
        max_features=None,
        min_samples_leaf=4,
        min_samples_split=2,
        random_state=42,
        splitter='best'))
])

pipeline_rf = Pipeline([
    ('scaler', StandardScaler()),
    ('rf', RandomForestRegressor(n_estimators=100, random_state=42))
])

pipeline_svr = Pipeline([
    ('scaler', StandardScaler()),
    ('svr', SVR(C=10,
        degree=3,
        epsilon=0.2,
        gamma=1,
        kernel='poly'))
])

# Voting Regressor
voting_reg = VotingRegressor(estimators=[
    #('lr', pipeline_lr),
    #('ridge', pipeline_ridge),
    #('lasso', pipeline_lasso),
    #('knn', pipeline_knn),
    ('tree', pipeline_tree),
    ('rf', pipeline_rf),
    ('svr', pipeline_svr)
])

# Train
voting_reg.fit(X_train, y_train)

VotingRegressor(estimators=[('tree',
                             Pipeline(steps=[('scaler', StandardScaler()),
                                             ('tree',
                                              DecisionTreeRegressor(max_depth=5,
                                                                    min_samples_leaf=4,
                                                                    random_state=42))])),
                            ('rf',
                             Pipeline(steps=[('scaler', StandardScaler()),
                                             ('rf',
                                              RandomForestRegressor(random_state=42))])),
                            ('svr',
                             Pipeline(steps=[('scaler', StandardScaler()),
                                             ('svr',
                                              SVR(C=10, epsilon=0.2, gamma=1,
                                                  kernel='poly'))]))])

In [ ]:
for name, reg in voting_reg.named_estimators_.items():
  print(name, "=", reg.score(X_test, y_test))

knn = 0.8276139187631248
tree = 0.8594369540424764
rf = 0.854386281612938
svr = 0.8513880078920127


In [ ]:
voting_reg.score(X_test, y_test)

0.8738310638147806

In [ ]:
y_pred = voting_reg.predict(X_test)

mae = mean_absolute_error(y_test , y_pred)
mse = mean_squared_error(y_test , y_pred)
rmse = np.sqrt(mse)
r2=r2_score(y_test , y_pred)

print(f"MAE: {mae}")
print(f"MSE: {mse}")
print(f"RMSE: {rmse}")
print(f"R2: {r2}")
print(100*'_')

MAE: 2145.697029322857
MSE: 18499433.08674511
RMSE: 4301.096730689175
R2: 0.8738310638147806
____________________________________________________________________________________________________


In [ ]:
voting_reg.voting = "soft"
voting_reg.named_estimators["svr"].probability = True
voting_reg.fit(X_train, y_train)
voting_reg.score(X_test, y_test)

0.8726665713316603

In [ ]:
y_pred = voting_reg.predict(X_test)

mae = mean_absolute_error(y_test , y_pred)
mse = mean_squared_error(y_test , y_pred)
rmse = np.sqrt(mse)
r2=r2_score(y_test , y_pred)

print(f"MAE: {mae}")
print(f"MSE: {mse}")
print(f"RMSE: {rmse}")
print(f"R2: {r2}")
print(100*'_')

MAE: 2245.7608100758325
MSE: 18670175.99243052
RMSE: 4320.899905393611
R2: 0.8726665713316603
____________________________________________________________________________________________________
